In [0]:
from google.colab import drive
drive.mount('/content/drive')

#Package

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
import seaborn as sns

import torch as T
import torch.nn as nn
import torch.nn.functional as F
import copy
import pandas as pd
from torch.autograd import Variable


#Config

In [0]:
max_enc_length = 2000     #99% of the data sequences are below this limit

data_path = 'drive/My Drive/Bioinformatic'
classes = ['cyto',  'secreted','mito', 'nucleus']
test_name = 'blind'
pkl_path = 'drive/My Drive/Bioinformatic/pkl'



#Pre-processing

##Load .fasta data

In [0]:
def _parse_fasta(data, lines, c):
    lines_iter = iter(lines)
    line = next(lines_iter, None)       #return None if there is no more items
    max_len = 0
    while line:
        assert line.startswith('>')
        info = line
        line = next(lines_iter, None)
        seq = ''
        while line and not line.startswith('>'):
            seq += line.rstrip()
            line = next(lines_iter, None)
        max_len = max((max_len, len(seq)))
        data['info'].append(info)
        data['seq'].append(seq)
        data['class'].append(c)
    return max_len

def _load_data(data, name, c):
    with open('{}/{}.fasta'.format(data_path, name), 'r') as f:
        lines = f.readlines()
    return _parse_fasta(data, lines, c)


def _get_data():
    train = {'info': [], 'seq': [], 'class': []}
    test = {'info': [], 'seq': [], 'class': []}
    max_len = _load_data(test, test_name, None)
    print('Max sequence length test: {}'.format(max_len))
    for c in classes:
        seq_len = _load_data(train, c, c)
        max_len = max((max_len, seq_len))
    print('Max sequence length: {}'.format(max_len))
    print('Train sequences: {}'.format(len(train['info'])))
    print('Test sequences: {}'.format(len(test['info'])))
    return train, test, max_len

train, test, max_len = _get_data()

##Build dictionary 

In [0]:



def build_dictionary(sentences, vocab=None, max_sent_len_=None):            #use later

    '''
    This function takes as input raw amino acid sequences and outputs
        - a dictionary and reverse ditionary containing all tokens (amino acids)
        - encoded amino acid sequences. This is because the RNN requires numeric inputs
        - the sequence length for each protein and the max sequence length across all proteins.
            This is needed for the dynamic RNN function in tf.
    If no vocabulary is provided, a new one will be create (this is used on the training data)
    If a vocabulary is provided, then this vocabulary will be used to encode sequences (this is used on the test data)
    '''
    is_ext_vocab = True

    # If no vocab provided, create a new one
    if vocab is None:
        is_ext_vocab = False
        vocab = {'<PAD>': 0, '<OOV>': 1}

    # Create list that will contain integer encoded senteces 
    data_sentences = []
    max_sent_len = -1

    for sentence in sentences:
        words = []
        for word in sentence:
            # If creating a new vocab, and word isnt in vocab yet, add it
            if not is_ext_vocab and word not in vocab:
                vocab[word] = len(vocab)
            # Now add either OOV or actual token to words
            if word not in vocab:
                token_id = vocab['<OOV>']       #word is not in the vocab
            else:
                token_id = vocab[word]
            words.append(token_id)
        if len(words) > max_sent_len:
            max_sent_len = len(words)
        data_sentences.append(words)

    if max_sent_len_ is not None:
        max_sent_len = max_sent_len_            #if max length has been setup before

    enc_sentences = np.full([len(data_sentences), max_sent_len], vocab['<PAD>'], dtype=np.int32)        #matrix full with id of <PAD>

    sentence_lengths = []
    for i, sentence in enumerate(data_sentences):
        enc_sentences[i, 0:len(sentence)] = sentence
        sentence_lengths.append(len(sentence))

    sentence_lengths = np.array(sentence_lengths, dtype=np.int32)
    reverse_dictionary = dict(zip(vocab.values(), vocab.keys()))

    return vocab, reverse_dictionary, sentence_lengths, max_sent_len+1, enc_sentences



def generate_pkl(train):

    '''
    This takes as input a train/test flag, and then takes the raw 
        input files in fasta format and return pickle files.
    '''
    if train:
        files = ['cyto.fasta', 'secreted.fasta', 'mito.fasta', 'nucleus.fasta']
        prefix = 'train_'
    else:
        files = ['blind.fasta']
        prefix = 'test_'

    data = []
    labels = []
    s = ''
    for fasta_file in files:                    #read each data file seperately
        with open('{}/{}'.format(data_path,fasta_file), 'r') as f:
            lines = f.readlines()
        count = 0
        for l in lines:
            if l[0] == '>':
                if count > 0:
                    data.append(s)
                    labels.append(files.index(fasta_file))         #label list ----0: cyto; 1: secreted;  2: mito; 3: nucleus
                s = ''
            else:
                s += l[:-1]
                if count == len(lines) - 1:
                    data.append(s)
                    labels.append(files.index(fasta_file))
            count += 1
    with open('{}/{}'.format(pkl_path,prefix + 'data.pkl'), 'wb') as f:
        pkl.dump(data, f)
    with open('{}/{}'.format(pkl_path,prefix + 'labels.pkl'), 'wb') as f:
        pkl.dump(labels, f)


def get_data(train):
    if train:
        prefix = 'train_'
    else:
        prefix = 'test_'
    if not os.path.exists(prefix + 'data.pkl'):
        print('Generating data')
        generate_pkl(train)
    with open('{}/{}'.format(pkl_path,prefix + 'data.pkl'), 'rb') as f:
        data = pkl.load(f)
    with open('{}/{}'.format(pkl_path,prefix + 'labels.pkl'), 'rb') as f:
        labels = pkl.load(f)
    return data, labels





##Save pickle files

In [0]:
Data, Labels = get_data(train = True)           #list with len = 9222

In [0]:
test_data, test_labels = get_data(train = False)        #testing

##Load pickle files

In [0]:
with open('{}/{}'.format(pkl_path,'train_' + 'data.pkl'), 'rb') as f:
    Data = pkl.load(f)
with open('{}/{}'.format(pkl_path,'train_' + 'labels.pkl'), 'rb') as f:
    Labels = pkl.load(f)

In [0]:
with open('{}/{}'.format(pkl_path, 'test_' + 'data.pkl'), 'rb') as f:
    test_data = pkl.load(f)
    #no label for testing data

##Dict

In [0]:
vocab, reverse_dictionary, sentence_lengths, max_sent_len, enc_sentences = build_dictionary(Data)       #training data, the enc_sentence has (9222,13100)
test_vocab, test_reverse, test_sentence_lengths, test_max_sent_len, test_enc_sentences = build_dictionary(test_data, vocab=vocab, max_sent_len_= max_enc_length)

##Statistics of data

In [0]:


def summary_stats(lengths, labels, name):

    '''
    Takes as input the lengths and labels of amino acid sequences
    Prints and returns pandas dataframes containing descriptive statistics
    '''

    bins = [0,100,500,1000,1500,2000,2499]
    labels_string = ['cyto', 'secreted', 'mito', 'nucleus']
        
    df = pd.DataFrame({'length': lengths, 'label': labels})
    table = pd.crosstab(np.digitize(df.length, bins), df.label)
    table.index = pd.Index(['[0, 100)', '[100, 500)', '[500, 1000]', '[1000, 1500)', '[1500, 2000)','[2000, 2500]', '[2500, inf]'], name="Bin")
    table.columns = pd.Index(labels_string, name="Class")
        
    sum_row = {col: table[col].sum() for col in table}
    sum_df = pd.DataFrame(sum_row, index=["Total"])
    table = table.append(sum_df)
    table['Total'] = table.sum(axis=1)

    print('\n~~~~~~~ Summary stats for %s set ~~~~~~~' % name)
    print('\nCount of sequence lengths by class')
    print(table)
    print('\nDescriptive statistics')
    print(df.describe())

    return df, table

In [0]:
plt.hist(test_sentence_lengths, bins = 20, range=(0,2000));
plt.xlabel('Length of sequences');
plt.ylabel('Frequency');
plt.title('Histogram of testing sequence length');


In [0]:
train_statistics, train_frame = summary_stats(sentence_lengths,np.array(Labels) ,'train')

In [0]:
valid_statistics, valid_frame = summary_stats(enc_valid_length, enc_valid_label, 'validation')

#Model

##LSTM, attention

In [0]:


class attentionLSTM(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_size, bidirection, if_attention):
        super(attentionLSTM, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.bidirection = bidirection
        self.if_attention = if_attention

        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)

        self.h_0 = nn.Parameter(T.rand(2 if self.bidirection else 1, self.batch_size, self.hidden_size).type(T.FloatTensor), requires_grad = True)  #learn initial state
        self.c_0 = nn.Parameter(T.rand(2 if self.bidirection else 1, self.batch_size, self.hidden_size).type(T.FloatTensor), requires_grad = True)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, batch_first = False, bidirectional = self.bidirection)
        self.linear1 = nn.Linear(self.hidden_size * (2 if self.bidirection else 1) , self.hidden_size )               #since we do for each batch data?
        #self.linear2 = nn.Linear(max_enc_length , self.output_size)
        self.linear2 = nn.Linear(self.hidden_size, self.output_size)

        self.att_wh = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), self.hidden_size * (2 if self.bidirection else 1), bias=False)
        self.att_ws = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), self.hidden_size * (2 if self.bidirection else 1))
        self.att_v = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), 1, bias = False)

    def attention_net(self, output_lstm, last_hidden, enc_padding_mask):
        """
        the attention of each of the hidden state of lstm and the last hidden state

        param: last_hidden : [num_direction, batch_size, hidden_size]
        param: output_lstm : [batch_size, seq_length(2500), num_direction * hidden_size]
        param: enc_padding_mask: [batch_size, seq_length(2500)], 
        """
        #hidden = last_hidden#.squeeze(0)                     #[batch_size, hidden_size]
        #hidden = last_hidden.view(last_hidden.shape[1], -1)

        #attention_weight = T.bmm(output_lstm, hidden.unsqueeze(2)).squeeze(2)           #[batch_size, seq_length], maybe do 0out probability of padded tokens??
        #attention_weight = attention_weight * enc_padding_mask
        #normalisation_factor = attention_weight.sum(1, keepdim = True)
        #attention_weight = attention_weight/normalisation_factor
        
        #softmax_att_weight = F.softmax(attention_weight,dim = 1)        #can try linear normalisation, if we zero out padding then cannot use softmax anymore
        
        #new_hidden_state = T.bmm(output_lstm.transpose(1,2), attention_weight.unsqueeze(2)).squeeze(2)        #[batch_size, num_direction * hidden_size]
        et = self.att_wh(output_lstm)           #[batch_size, seq_length, num_direction*hidden_size]
        hidden = last_hidden.view(last_hidden.shape[1], -1)         #[batch_size, num_di*hidden_size]
        final_fea = self.att_ws(hidden).unsqueeze(1)                #[batch_size, 1, num_di*hidden_size]
        et = et + final_fea
        et = T.tanh(et)
        et = self.att_v(et).squeeze(2)          #[batch_size, seq_length]

        et1 = F.softmax(et, dim = 1)
        at = et1 * enc_padding_mask
        normalisation_factor = at.sum(1, keepdim = True)
        at = at/normalisation_factor

        at = at.unsqueeze(1)            #[batch_size, 1, seq_length]
        ct_e = T.bmm(at, output_lstm)      #[batch_size, 1, num_direction * hidden_size]
        ct_e = ct_e.squeeze(1)
        at = at.squeeze(1)
        return ct_e, at    #(batch_size, num_direction * hidden_size)


    def forward(self,input_sequence, enc_padding_mask,batch_size_setup = None):

        embedded = self.embedding(input_sequence)   #input_seq [batch_size, seq_length]; embedded [batch_size, seq_length, embedding_size]
        embedded = embedded.permute(1,0,2)          #so that no need to batch_first;  [seq_length, batch_size, embedding_size]  https://discuss.pytorch.org/t/could-someone-explain-batch-first-true-in-lstm/15402

        output, (final_hidden_state, final_cell_state) = self.lstm(embedded, (self.h_0, self.c_0))  #output [seq_length, batch, nu_direction*hidden_szie]
        output = output.permute(1,0,2)                          #[batch_size, seq_length, num_di*hidden_size]          

        if self.if_attention:
            attention_output, attention_weight= self.attention_net(output, final_hidden_state, enc_padding_mask)       #[batch_size, num_direction * hidden_size]
            output = self.linear1(attention_output)             #[batch_size,  hidden_size]
            output = T.tanh(output)
            logits = F.softmax(self.linear2(output),dim=1)
        else:      #no attention
            output = output.permute(0, 2, 1)            #[batch_size, num_di*hidden_size, seq_length]
            output = F.max_pool1d(output, output.shape[2]).squeeze(2)   #[batch_size, num_di*hidden_size],  choose maximum value at each hidden entry
            output = self.linear1(T.tanh(output))
            logits = F.softmax(self.linear2(output), dim = 1)       # MLP

            #output = self.linear1(final_hidden_state.squeeze(1))
            #output = T.tanh(output)
            #output = self.linear2(output)
            #logits = F.softmax(output, dim = 1)

            """
            output = T.tanh(output)                                         #[batch_size, seq_length, num_direction*hidden_size]
            output = F.max_pool1d(output, output.shape[2]).squeeze(2)       #[batch_size, seq_length]
            output = T.tanh(output)
            output = output * enc_padding_mask                              #[batch_size, seq_length]
            logits = F.softmax(self.linear2(output), dim = 1)                #[batch_size, output_size]
            """
            #logits = F.softmax(self.linear(final_hidden_state[-1]), dim = 1)
            attention_weight = 0

        return logits, attention_weight

##LSTM, CNN

In [0]:
class CNN_lstm(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_size, kernel_size,bidirection, if_attention):
        super(CNN_lstm, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.kernel_size = kernel_size
        self.num_kernel = len(kernel_size)
        self.bidirection = bidirection
        self.if_attention = if_attention

        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)

        KK = []
        for k in kernel_size:
            KK.append( k + 1 if k % 2 == 0 else k)
        self.conv = [get_cuda(nn.Conv2d(1, self.embedding_size, (k, self.embedding_size), stride = 1, padding = (k // 2, 0))) for k in KK]

        #self.conv = nn.Conv2d(1, self.embedding_size, (self.kernel_size, self.embedding_size), stride=1, padding=(1, 0))

        self.h_0 = nn.Parameter(T.rand(2 if self.bidirection else 1, self.batch_size, self.hidden_size).type(T.FloatTensor), requires_grad = True)  #learn initial state
        self.c_0 = nn.Parameter(T.rand(2 if self.bidirection else 1, self.batch_size, self.hidden_size).type(T.FloatTensor), requires_grad = True)
        self.lstm = nn.LSTM(self.embedding_size * self.num_kernel, self.hidden_size, batch_first = False, bidirectional = self.bidirection)
        self.linear1 = nn.Linear(self.hidden_size * (2 if self.bidirection else 1) , (self.hidden_size * (2 if self.bidirection else 1))//2)               #since we do for each batch data?
        self.linear2 = nn.Linear((self.hidden_size * (2 if self.bidirection else 1))//2, self.output_size )

        self.relu = nn.ReLU()
        #self.linear2 = nn.Linear(max_enc_length , self.output_size)

        self.att_wh = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), self.hidden_size * (2 if self.bidirection else 1), bias=False)
        self.att_ws = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), self.hidden_size * (2 if self.bidirection else 1))
        self.att_v = nn.Linear(self.hidden_size * (2 if self.bidirection else 1), 1, bias = False)
    
    def attention_net(self, output_lstm, last_hidden, enc_padding_mask):
        """
        the attention of each of the hidden state of lstm and the last hidden state

        param: last_hidden : [num_direction, batch_size, hidden_size]
        param: output_lstm : [batch_size, seq_length(2500), num_direction * hidden_size]
        param: enc_padding_mask: [batch_size, seq_length(2500)], 
        """
        et = self.att_wh(output_lstm)           #[batch_size, seq_length, num_direction*hidden_size]
        hidden = last_hidden.view(last_hidden.shape[1], -1)         #[batch_size, num_di*hidden_size]
        final_fea = self.att_ws(hidden).unsqueeze(1)                #[batch_size, 1, num_di*hidden_size]
        et = et + final_fea
        et = T.tanh(et)
        et = self.att_v(et).squeeze(2)          #[batch_size, seq_length]

        et1 = F.softmax(et, dim = 1)
        at = et1 * enc_padding_mask
        normalisation_factor = at.sum(1, keepdim = True)
        at = at/normalisation_factor

        at = at.unsqueeze(1)            #[batch_size, 1, seq_length]
        ct_e = T.bmm(at, output_lstm)      #[batch_size, 1, num_direction * hidden_size]
        ct_e = ct_e.squeeze(1)
        at = at.squeeze(1)

        return ct_e, at

        """
        hidden = last_hidden.view(last_hidden.shape[1], -1)
        print('hidden size', hidden.size())

        attention_weight = T.bmm(output_lstm, hidden.unsqueeze(2)).squeeze(2)           #[batch_size, seq_length], maybe do 0out probability of padded tokens??
        print('attention weight size', attention_weight.size())
        attention_weight = attention_weight * enc_padding_mask
        normalisation_factor = attention_weight.sum(1, keepdim = True)
        attention_weight = attention_weight/normalisation_factor
        
        #softmax_att_weight = F.softmax(attention_weight,dim = 1)        #can try linear normalisation, if we zero out padding then cannot use softmax anymore
        
        new_hidden_state = T.bmm(output_lstm.transpose(1,2), attention_weight.unsqueeze(2)).squeeze(2)        #[batch_size, num_direction * hidden_size]

        return new_hidden_state, attention_weight     #(batch_size, num_direction * hidden_size)
        """

    def forward(self, input_sequence, enc_padding_mask): 
        """
        input_sequence.size() :  [batch_size, seq_length]
        """
        embedded = self.embedding(input_sequence)        #embedded.size(): [batch_size, seq_length, embedding_size]
        embedded = embedded.unsqueeze(1)

        conved = [self.relu(conv(embedded)).squeeze(-1) for conv in self.conv]        #each entry has [batch_size, embedding_size, seq_length]
        conved = T.cat(conved, 1)               #concatenate, [batch_size, num_kernel*embedding_size, seq_length]
        conved = conved.permute(2, 0, 1)        #[seq_length, batch_size, num_kernel*embedding_size]

        output, (final_hidden_state, final_cell_state) = self.lstm(conved, (self.h_0, self.c_0))     #[seq_length, batch_size, num_di*hidden_size]
        output = output.permute(1,0,2)                          #[batch_size, seq_length, num_di*hidden_size]


        """
        conved = self.conv(embedded.unsqueeze(1))        #conved.size():  [batch_size, embedding_size, seq_length, 1]
        #print('size of conved 1: ', conved.size())
        conved = conved.squeeze(-1).permute(2, 0, 1)     #                [seq_length, batch_size, embedding_size]

        #print('size of conved:', conved.size())

        output, (final_hidden_state, final_cell_state) = self.lstm(conved, (self.h_0, self.c_0))        #output.size():  [seq_length, batch, nu_direction*hidden_size]
        output = output.permute(1,0,2)                  # [batch_size, seq_length, num_direction*hidden_size]
        """
        if self.if_attention:
            attention_output, attention_weight= self.attention_net(output, final_hidden_state, enc_padding_mask)       #[batch_size, num_direction * hidden_size]
            logits = F.softmax(self.linear1(attention_output),dim=1)
        else:      #no attention

            output = output.permute(0, 2, 1)     
            #print('ouytput size', output.size())                                   #[batch_size, num_di*hidden_size, seq_length]
            output = F.max_pool1d(output, output.shape[2]).squeeze(2)               #[batch_size, num_di * hidden_size]
            output = T.tanh(output)
            output = self.linear1(output)
            output = T.tanh(output)
            logits = F.softmax(self.linear2(output), dim = 1)                        #this version is remove the seq_length dimension and no padding mask is involved

            """
            output = T.tanh(output)                                         #[batch_size, seq_length, num_direction*hidden_size]
            output = F.max_pool1d(output, output.shape[2]).squeeze(2)       #[batch_size, seq_length]
            output = T.tanh(output)
            output = output * enc_padding_mask                              #[batch_size, seq_length]
            logits = F.softmax(self.linear2(output), dim = 1)                #[batch_size, output_size]
            """
            #logits = F.softmax(self.linear(final_hidden_state[-1]), dim = 1)
            attention_weight = 0

        return logits, attention_weight


#Run (training, validation, testing)

In [0]:


def gradient_clamping(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def data_shuffle(enc_seq, enc_label, enc_length,random_seed = None):
    if random_seed is not None:
        np.random.seed(random_seed)
    perm_rnd = np.random.permutation(len(enc_seq))
    perm_seq = enc_seq[perm_rnd]
    perm_label = enc_label[perm_rnd]
    perm_length = enc_length[perm_rnd]

    return perm_seq, perm_label, perm_length

def truncate(enc_sentences, sentence_lengths):
    original_length = copy.copy(sentence_lengths)
    enc_truncated = enc_sentences[:,:max_enc_length]
    for i in range(len(sentence_lengths)):
        if sentence_lengths[i] > max_enc_length:
            enc_truncated[i] = np.concatenate((enc_sentences[i,:max_enc_length - 100], enc_sentences[i,-100:]), axis = 0)
            sentence_lengths[i] = max_enc_length    #truncated length of enc_sentences
    #enc_labels = np.array(Labels)       #into numpy array
    return enc_truncated, sentence_lengths


class run():
    def __init__(self, vocab,batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gradient_clip, kernel_size=0):
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.kernel_size = kernel_size
        self.num_epoch = num_epoch
        self.lr = lr
        self.gradient_clip = gradient_clip

        self.train_loss = []
        self.train_acc = []
        self.valid_loss = []
        self.valid_acc = []
        self.count_matrix = np.zeros((4,4))

        self.attention_dict = {}
        self.data_att_dict = {}

        #testing
        self.prediction_score = 0
        self.att_test = 0
        self.vocab = vocab

    
    def setup_model(self, bidirection, if_attention, model_name):
        if model_name == 'CNN-lstm':
            self.model = CNN_lstm(self.batch_size, self.output_size, self.hidden_size, self.vocab_size, self.embedding_size, self.kernel_size, bidirection, if_attention)
        else:
            self.model = attentionLSTM(self.batch_size, self.output_size, self.hidden_size, self.vocab_size, self.embedding_size, bidirection, if_attention)
        self.model = get_cuda(self.model)
        self.optimiser = T.optim.Adam(self.model.parameters(), lr = self.lr)
        pass
    
    def save_model(self, save_model_path, epoch):
        save_path = save_model_path + "/%05d.tar"%(epoch+1)
        T.save({
            'epoch': epoch+1,
            'model_dict': self.model.state_dict(),
            'optimiser_dict': self.optimiser.state_dict()
        }, save_path)

    '''
    def data_shuffle(self, enc_seq, enc_label, enc_length,random_seed = None):
        if random_seed is not None:
            np.random.seed(random_seed)
        perm_rnd = np.random.permutation(len(enc_seq))
        perm_seq = enc_seq[perm_rnd]
        perm_label = enc_label[perm_rnd]
        perm_length = enc_length[perm_rnd]

        return perm_seq, perm_label, perm_length
    '''
    def one_batcher(self, seq, label, length, batch_index):
        """
        params: batch_index: the index of current batch
        """
        #if (batch_index+1) * self.batch_size > seq.shape[0]:        #excess the index of array
        #    batch_data = seq[batch_index * self.batch_size :]
        #    batch_label = label[batch_index * self.batch_size :]
        #    batch_length = length[batch_index * self.batch_size :]
        #else:
        batch_data = seq[batch_indedx * self.batch_size : (batch_index + 1) * self.batch_size]
        batch_label = label[batch_index * self.batch_size : (batch_index + 1) * self.batch_size]
        batch_length = length[batch_index * self.batch_size : (batch_index + 1) * self.batch_size]
        #fill in the padding array
        enc_padding_mask = np.zeros(batch_data.shape, dtype = np.float32)
        for row in range(batch_data.shape[0]):
            enc_padding_mask[row, :batch_length[row]] = np.ones(batch_length[row], dtype = np.float32)
           
        return batch_data, batch_label, batch_length, enc_padding_mask


    def train(self, enc_train, enc_train_label, enc_train_length, epoch):
        """
        training for each epoch, first mini-batching, then train the model
        params: enc_train: the encoded training sequence 
        params: enc_train_label: the encoded labels 
        params: enc_train_length: the corresponding length for the encoded sequence 
        """

        counter = 0
        total_epoch_loss = 0
        total_epoch_acc = 0
        train_perm, train_label_perm, train_length_perm = data_shuffle(enc_train, enc_train_label, enc_train_length, random_seed = epoch+1)      #random shuffling the trainig data
        #mini batching
        N = enc_train.shape[0]
        for n in range(N // self.batch_size):
            batch_data, batch_label, _, enc_padding_mask = self.one_batcher(train_perm, train_label_perm, train_length_perm, batch_index = n)
            input_data = T.LongTensor(batch_data)
            input_label = T.LongTensor(batch_label)
            input_padding_mask = T.LongTensor(enc_padding_mask)

            input_data = get_cuda(input_data)
            input_label = get_cuda(input_label)
            input_padding_mask = get_cuda(input_padding_mask)

            self.optimiser.zero_grad()
            #print('size of input_data:', input_data.size())
            prediction_score,_ = self.model(input_data, enc_padding_mask = input_padding_mask)        #return [batch_size, output_size] logits
            loss = F.cross_entropy(prediction_score, input_label)

            #training accuracy
            num_correct = (T.max(prediction_score, 1)[1].view(input_label.size()).data == input_label.data).sum()
            acc = 100.0 * num_correct/(len(batch_data))                

            loss.backward()
            if self.gradient_clip:
                gradient_clamping(self.model, 1e-1)      #gradient clipping
            self.optimiser.step()
            counter+=1
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()
        #list_train_loss.append(total_epoch_loss/counter)
        #list_train_acc.append(total_epoch_acc/counter)
        #print('After {} epoch, the training loss is {:.4f}; training accuracy is {:.2f}%'.format(i+1, total_epoch_loss/counter, total_epoch_acc/counter))
        return total_epoch_loss/counter, total_epoch_acc/counter
    
    def validation(self, enc_valid, enc_valid_label, enc_valid_length, epoch, if_attention):
        """
        validating the model for each epoch
        """
        total_epoch_loss = 0
        total_epoch_acc = 0

        if epoch % 10 == 0:
            self.count_matrix = np.zeros((4,4))       #re-initialise error matrix

        attention_dict = {'0':[], '1':[], '2':[], '3':[]}     #0: cyto; 1: secreted; 2: Mito; 3: nucleus
        data_att_dict = {'0':[], '1':[], '2':[], '3':[]}

        enc_valid, enc_valid_label, enc_valid_length = data_shuffle(enc_valid, enc_valid_label, enc_valid_length, random_seed = epoch+10)  #shuffle validation data
        with T.no_grad():
            N = enc_valid.shape[0]
            #num_batch = N//self.batch_size if (N%self.batch_size==0) else (N//self.batch_size)+1
            num_batch = N//self.batch_size
            for n in range(num_batch):
                batch_valid_data, batch_valid_label, _, enc_padding_mask_valid = self.one_batcher(enc_valid, enc_valid_label, enc_valid_length, 
                                                                                                                   batch_index = n)
                input_valid_data = T.LongTensor(batch_valid_data)
                input_valid_label = T.LongTensor(batch_valid_label)
                input_valid_padding_mask = T.LongTensor(enc_padding_mask_valid)
                
                input_valid_data = get_cuda(input_valid_data)
                input_valid_label = get_cuda(input_valid_label)
                input_valid_padding_mask = get_cuda(input_valid_padding_mask)
                                                                                                            #need to setup batch size as we might have leftover batches
                prediction_valid_score, attention_weight= self.model(input_valid_data, enc_padding_mask = input_valid_padding_mask) #batch_size_setup = batch_valid_data.shape[0])
                loss_valid = F.cross_entropy(prediction_valid_score, input_valid_label)
                #validation accuracy
                num_correct_valid = (T.max(prediction_valid_score, 1)[1].view(input_valid_label.size()).data == input_valid_label.data).sum()
                acc_valid = 100.0 * num_correct_valid/(len(batch_valid_data))

                #update attention matrix
                if if_attention:
                    if epoch%10 == 0:
                        pred_label = T.max(prediction_valid_score, dim=1)[1]    #vector of predicted label
                        for i in range(len(pred_label)):
                            if int(pred_label[i]) == int(input_valid_label[i]):     #correct prediction
                                attention_dict[str(int(pred_label[i]))].append(attention_weight[i].unsqueeze(0))

                                #update data dictionary
                                data_att_dict[str(int(pred_label[i]))].append(input_valid_data[i,:].unsqueeze(0))


                        self.attention_dict = attention_dict
                        self.data_att_dict = data_att_dict

                #update error matrix
                if epoch % 10 ==0:
                    pred_temp = T.max(prediction_valid_score, 1)[1].view(input_valid_label.size()).data
                    for i in range(input_valid_label.shape[0]):
                        self.count_matrix[int(pred_temp[i]), int(input_valid_label.data[i])] += 1

                total_epoch_loss += loss_valid
                total_epoch_acc += acc_valid
        return total_epoch_loss/num_batch, total_epoch_acc/num_batch

    def lets_go(self, sentence_lengths, enc_truncated, enc_labels,k_fold, k,if_bidirection, if_attention, model_name,save_model_path=None):
        '''
        perform five-fold cross valdiation
        params: sentence_length: array contains the truncated lengh of enc_sentences
        params: enc_sentences: the encoded sequence with original length
        params: Labels: array contains the label of enc_sentences
        params: k_fold: K value of corss-validation
        params: valud of kth fold
        '''
        self.setup_model(bidirection = if_bidirection, if_attention = if_attention, model_name = model_name)
        #enc_labels = np.array(Labels)

        val_size = len(sentence_lengths) // k_fold      #size of each segment
        #start splitting the data
        val_mask = np.arange(val_size * (k-1), val_size * k)        #kth fold
        enc_valid = enc_truncated[val_mask]
        enc_valid_label = enc_labels[val_mask]
        enc_valid_length = sentence_lengths[val_mask]

        enc_train = np.delete(enc_truncated, val_mask, axis = 0)
        enc_train_label = np.delete(enc_labels, val_mask, axis = 0)
        enc_train_length = np.delete(sentence_lengths, val_mask, axis = 0)
        #keep the training and validation data fixed
        for epoch in range(self.num_epoch):

            epoch_train_loss, epoch_train_acc = self.train(enc_train, enc_train_label, enc_train_length, epoch)
            epoch_valid_loss, epoch_valid_acc = self.validation(enc_valid, enc_valid_label, enc_valid_length, epoch, if_attention)

            self.train_loss.append(epoch_train_loss)
            self.train_acc.append(epoch_train_acc)
            self.valid_loss.append(epoch_valid_loss)
            self.valid_acc.append(epoch_valid_acc)
            print(f'Epoch:{epoch+1 :02}, Train loss: {epoch_train_loss :.3f}, Train acc: {epoch_train_acc :.2f}%, Val loss: {epoch_valid_loss :.3f}, Val acc: {epoch_valid_acc :.2f}%')
            
            #if save_model_path is not None:
            #    if (epoch+1)%5 == 0:
            #        self.save_model(save_model_path, epoch)


        return self.train_loss, self.train_acc, self.valid_loss, self.valid_acc, self.count_matrix


        '''
        #at each epoch, perform data random shuffling followed by K-fold cross-validation and train and validate on each combination
        for epoch in range(self.num_epoch):
            rnd_enc_seq, rnd_label, rnd_length = self.data_shuffle(enc_truncated, enc_labels, sentence_lengths, epoch+5)     #random shuffling
            epoch_train_loss = 0
            epoch_train_acc = 0
            epoch_valid_loss = 0
            epoch_valid_acc = 0

            val_mask = np.arange(val_size * k, val_size * (k+1))        #[0:val]-->[val:2*val]-->[2*val:3*val]....

            enc_valid = rnd_enc_seq[val_mask]                   #validation data splitting
            enc_valid_label = rnd_label[val_mask]
            enc_valid_length = rnd_length[val_mask]

            enc_train = np.delete(rnd_enc_seq, val_mask, axis = 0)          #training data splitting
            enc_train_label = np.delete(rnd_label, val_mask, axis = 0)
                enc_train_length = np.delete(rnd_length, val_mask, axis = 0)

                k_train_loss, k_train_acc = self.train(enc_train, enc_train_label, enc_train_length)         #perform training
                k_valid_loss, k_valid_acc = self.validation(enc_valid, enc_valid_label, enc_valid_length)    #perform validating

                epoch_train_loss += k_train_loss
                epoch_train_acc += k_train_acc
                epoch_valid_loss += k_valid_loss
                epoch_valid_acc += k_valid_acc
            
            current_epoch_train_loss = epoch_train_loss/k_fold      #take the average
            current_epoch_train_acc = epoch_train_acc/k_fold
            current_epoch_valid_loss = epoch_valid_loss/k_fold
            current_epoch_valid_acc = epoch_valid_acc/k_fold

            print(f'Epoch:{epoch+1 :02}, Train loss: {current_epoch_train_loss :.3f}, Train acc: {current_epoch_train_acc :.2f}%, Val loss: {current_epoch_valid_loss :.3f}, Val acc: {current_epoch_valid_acc :.2f}%')
            self.train_loss.append(current_epoch_train_loss)
            self.train_acc.append(current_epoch_train_acc)
            self.valid_loss.append(current_epoch_valid_loss)
            self.valid_acc.append(current_epoch_valid_acc)

            if save_model_path is not None:
                if (epoch+1)%5 == 0:
                    self.save_model(save_model_path, epoch)

        return self.train_loss, self.train_acc, self.valid_loss, self.valid_acc
        '''
    def train_full_data(self, sentence_lengths, enc_truncated, enc_labels, if_bidirection, if_attention, model_name,save_model_path=None):
        """
        performing training on full input data 
        """
        self.setup_model(bidirection = if_bidirection, if_attention = if_attention, model_name = model_name)
        enc_train = enc_truncated
        enc_train_label = enc_labels
        enc_train_length = sentence_lengths

        for epoch in range(self.num_epoch):
            epoch_train_loss, epoch_train_acc = self.train(enc_train, enc_train_label, enc_train_length, epoch)

            self.train_loss.append(epoch_train_loss)
            self.train_acc.append(epoch_train_acc)
            
            print(f'Epoch:{epoch+1 :02}, Train loss: {epoch_train_loss :.3f}, Train acc: {epoch_train_acc :.2f}%.')

        return self.train_loss, self.train_acc

    def testing(self,testing_enc_seq, testing_length):
        """
        perform testing, first padd the sequence if it has fewer samples than batch size
        test_enc_seq:  array containing encoded testing sequence
        testing_length:  array containing original length of the testin sequence
        """

        N = testing_enc_seq.shape[0]

        batch_test, test_padding_mask = self.test_padding(testing_enc_seq, testing_length)
        assert batch_test.shape == test_padding_mask.shape == (self.batch_size, max_enc_length) 

        input_test = T.LongTensor(batch_test)
        input_test_padding_mask = T.LongTensor(test_padding_mask)

        input_test = get_cuda(input_test)
        input_test_padding_mask = get_cuda(input_test_padding_mask)

        pred, att_w = self.model(input_test, input_test_padding_mask)
        self.prediction_score = pred[:N]            #discard the padding entry
        self.att_test = att_w[:N]

        return self.att_test            #outputing testing attention weight matrix 

    def test_padding(self, input_seq, input_length):
        """
        pad the testing data as it has fewer samples than the batch size
        input_seq:  encoded testing sequence
        input_length:  the original length of testing sequence
        """
            
        padded = np.full((self.batch_size, input_seq.shape[1]), self.vocab['<PAD>'] ,dtype = np.float32)
        padded[:input_seq.shape[0], :] = input_seq
        enc_padding_mask = np.full(padded.shape, self.vocab['<PAD>'], dtype = np.float32)
        for row in range(len(input_length)):    #20
            enc_padding_mask[row, :input_length[row]] = np.ones(input_length[row], dtype = np.float32)
        enc_padding_mask[len(input_length):, 1] = np.ones(self.batch_size - len(input_length), dtype = np.float32)
        
        return padded, enc_padding_mask

    def print_frame(self):
        """
        print the dataframe of predictive probability of each of four classes (testing)
        """
        prob = self.prediction_score.detach().cpu().numpy()
        prob_max = T.max(self.prediction_score, axis = 1)[1].cpu().numpy()
        pred_class = np.array([])
        for i in prob_max:
            pred_class = np.append(pred_class, classes[i])

        test_label_frame = []
        for j in test['info']:
            test_label_frame.append(j[1:-1])

        df = pd.DataFrame(prob, index = test_label_frame,columns = classes)
        df = df.round(3)
        df['Prediction'] = pd.Series(pred_class, index = df.index)

        return df



    def plot(self, mode):
        if mode == 'loss':
            plt.plot(self.train_loss, label = 'training loss')
            plt.plot(self.valid_loss, label = 'validation loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss plot')
            plt.legend()

        if mode == 'acc':
            plt.plot(self.train_acc, label = 'training acc')
            plt.plot(self.valid_acc, label = 'validation acc')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title('Accuracy plot')
            plt.legend()

    def matrix_plot(self):
        fig, ax = plt.subplots(1,2, figsize=(10,10))

        xlabel_list = [' ','cyto', ' ','secreted', ' ','mito', ' ','nucleus']
        ylabel_list = [' ','cyto', ' ','secreted', ' ','mito', ' ','nucleus']
        img = ax[0].imshow(self.count_matrix, aspect = 1, cmap = 'Blues')
        ax[0].set_xticklabels(xlabel_list)
        ax[0].set_yticklabels(ylabel_list)
        ax[0].set_xlabel('prediction')
        ax[0].set_ylabel('Ground truth')
        ax[0].set_title('Matrix plot during validation')

        for i in range(4):
            for j in range(4):
                c = self.count_matrix[j,i]
                ax[0].text(i, j, str(int(c)), va='center', ha='center')
        '''
        temp_matrix = copy.copy(self.count_matrix)
        np.fill_diagonal(temp_matrix, 0)
        img2 = ax[1].imshow(temp_matrix, aspect = 1,cmap = 'Blues')
        ax[1].set_xticklabels(xlabel_list)
        ax[1].set_yticklabels(ylabel_list)
        ax[1].set_xlabel('prediction')
        ax[1].set_ylabel('Ground truth')
        ax[1].set_title('Error during validation')   

        for i in range(4):
            for j in range(4):
                c = temp_matrix[j,i]
                ax[1].text(i, j, str(int(c)), va='center', ha='center')  
        '''
        row_sum = self.count_matrix.sum(axis = 0, keepdims = True)          #0 for precision, 1 for recall
        error_rate = self.count_matrix/row_sum
        img2 = ax[1].imshow(error_rate, aspect=1, cmap = 'Blues')
        ax[1].set_xticklabels(xlabel_list)
        ax[1].set_yticklabels(ylabel_list)
        ax[1].set_xlabel('prediction')
        ax[1].set_ylabel('Ground truth')
        ax[1].set_title('Error during validation')        

        for i in range(4):
            for j in range(4):
                c = error_rate[j,i]
                ax[1].text(i,j, '{:.2f}%'.format(c*100), va = 'center', ha = 'center')
        

        fig.colorbar(img2,orientation='horizontal')
        plt.tight_layout()
    
    def attention(self):
        return self.attention_dict, self.data_att_dict



#Experiment

In [0]:

perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 12)
enc_truncated, enc_length = truncate(perm_seq, perm_length)     #fixed for each model training


##LSTM

In [0]:
batch_size = 32        #32,64,128,256
output_size = 4
hidden_size = 128        #128 for CNN-lstm without attention
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [3, 5, 9]                 #for CNN+attention +LSTM the hidden size need to be lss
model_name = 'lstm'
num_epoch = 32
k_fold = 5                  #k-fold value
lr = 0.001
bidirection = False
if_attention = False
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


LSTM_1 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL1, Tacc1, VL1, Vacc1, count_matrix = LSTM_1.lets_go(enc_length, enc_truncated, perm_label, k_fold,1, bidirection, if_attention, model_name,save_model_path)


#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

In [0]:
LSTM_1.matrix_plot()

##Bi-LSTM

In [0]:
batch_size = 32        #32,64,128,256
output_size = 4
hidden_size = 128        #128 for CNN-lstm without attention
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [3, 5, 9]                 #for CNN+attention +LSTM the hidden size need to be lss
model_name = 'lstm'
num_epoch = 32
k_fold = 5                  #k-fold value
lr = 0.001
bidirection = True
if_attention = False
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


BLSTM = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL2, Tacc2, VL2, Vacc2, count_matrix2 = BLSTM.lets_go(enc_length, enc_truncated, perm_label, k_fold,1, bidirection, if_attention, model_name,save_model_path)


#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

In [0]:
BLSTM.matrix_plot()

##LSTM-Attention

In [0]:
batch_size = 32        #32,64,128,256
output_size = 4
hidden_size = 128        #128 for CNN-lstm without attention
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [3, 5, 9]                 #for CNN+attention +LSTM the hidden size need to be lss
model_name = 'lstm'
num_epoch = 22
k_fold = 5                  #k-fold value
lr = 0.001
bidirection = False
if_attention = True
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


LSTM_att = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL_att, Tacc_att, VL_att, Vacc_att, count_matrix_att = LSTM_att.lets_go(enc_length, enc_truncated, perm_label, k_fold,1, bidirection, if_attention, model_name,save_model_path)


#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

In [0]:
at, data_at = LSTM_att.attention()
data_se = T.cat(data_at['1']).cpu().numpy()
at_se = T.cat(at['1']).cpu().numpy()


In [0]:
seq_se = []
for i in range(data_se.shape[0]):
    temp = ''
    for j in range(data_se.shape[1]):
        if data_se[i,j] != 0:
            temp += reverse_dictionary[data_se[i,j]]
    seq_se.append(temp)
seq_se

In [0]:
plt.imshow(at_se, aspect='auto', cmap='binary', vmin=0, vmax=0.1)

In [0]:
peptide = []
threshold = 0.05
for i in range(at_se.shape[0]):
    temp = ''
    for j in range(at_se.shape[1]):
        if at_se[i,j] == 0:
            break
        if at_se[i,j] >threshold:
            temp += seq_se[i][j]
        else:
            temp += '_'
    peptide.append(temp)



In [0]:
for i in range(len(peptide)):
    print(seq_se[i] + '\n' + peptide[i])

In [0]:
t = 3
fig, ax = plt.subplots(2,2,figsize=(15,15))


ax[0,0].plot(at_se[t][:len(seq_se[t])], label='Secreted protein')
for i,txt in enumerate(seq_se[t]):
    ax[0,0].annotate(txt, (i, at_se[t][i]),fontsize='xx-large')
ax[0,0].set_xlabel('position')
ax[0,0].set_ylabel('attention weight');
ax[0,0].legend()

t2 = 6
ax[0,1].plot(at_se[t2][:len(seq_se[t2])], label='Secreted protein')
for i,txt in enumerate(seq_se[t2]):
    ax[0,1].annotate(txt, (i, at_se[t2][i]),fontsize='xx-large')
ax[0,1].set_xlabel('position')
ax[0,1].set_ylabel('attention weight');
ax[0,1].legend()

t3 = 8
ax[1,0].plot(at_se[t3][:len(seq_se[t3])], label='Secreted protein')
for i,txt in enumerate(seq_se[t3]):
    ax[1,0].annotate(txt, (i, at_se[t3][i]),fontsize='xx-large')
ax[1,0].set_xlabel('position')
ax[1,0].set_ylabel('attention weight');
ax[1,0].legend()

t4 = 203
ax[1,1].plot(at_se[t4][:len(seq_se[t4])], label='Secreted protein')
for i,txt in enumerate(seq_se[t4]):
    ax[1,1].annotate(txt, (i, at_se[t4][i]),fontsize='xx-large')
ax[1,1].set_xlabel('position')
ax[1,1].set_ylabel('attention weight');
ax[1,1].legend()

In [0]:
#attention plot

at_secreted = T.cat(at['1'], dim=0).cpu().numpy()
at_secreted

adjust_se = np.zeros(at_secreted.shape)

for i in range(at_secreted.shape[0]):
    num_nonzero = np.count_nonzero(at_secreted[i])
    if num_nonzero <2000:
        adjust_se[i,: int(num_nonzero/2)] = at_secreted[i, : int(num_nonzero/2)]
        adjust_se[i,-(num_nonzero - int(num_nonzero/2)):] = at_secreted[i,int(num_nonzero/2):num_nonzero]




In [0]:

fig, ax = plt.subplots(figsize=(10, 10))
#img = ax[0].imshow(adjust, interpolation='nearest',aspect='auto', cmap = 'binary', vmin=0, vmax=0.03);
img2 = ax.imshow(adjust_se, interpolation='nearest', aspect='auto', cmap = 'binary', vmin = 0, vmax = 0.03);
ax.set_title('Secreted attention')
ax.set_xlabel('Position')
ax.set_ylabel('Sequences number')
fig.colorbar(img2,orientation='vertical')

#CNN-LSTM

In [0]:
batch_size = 64        #32,64,128,256
output_size = 4
hidden_size = 128
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [15, 21, 27]
model_name = 'CNN-lstm'
num_epoch = 100
k_fold = 5
lr = 0.001
bidirection = False
if_attention = False
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


LSTM_cnn = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL4, Tacc4, VL4, Vacc4, count_matrix4 = LSTM_cnn.lets_go(enc_length, enc_truncated, perm_label, k_fold,1, bidirection, if_attention, model_name,save_model_path)


#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

In [0]:
LSTM_cnn.matrix_plot()

##5-folds cross validation using LSTM with attention

###fold-1

In [0]:
batch_size = 32        #32,64,128,256
output_size = 4
hidden_size = 128        #128 for CNN-lstm without attention
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [3, 5, 9]                 #for CNN+attention +LSTM the hidden size need to be lss
model_name = 'lstm'
num_epoch = 60
k_fold = 5                  #k-fold value
lr = 0.001
bidirection = False
if_attention = True
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


att1 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL1, Tacc1, VL1, Vacc1, count_matrix1 = att1.lets_go(enc_length, enc_truncated, perm_label, k_fold,1, bidirection, if_attention, model_name,save_model_path)
                                                                                                #fold 1!

#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

...........

###train on full data and perform testing

In [0]:
batch_size = 32        #32,64,128,256
output_size = 4
hidden_size = 128        #128 for CNN-lstm without attention
vocab_size = len(vocab)
embedding_size = 64
kernel_size = [3, 5, 9]                 #for CNN+attention +LSTM the hidden size need to be lss
model_name = 'lstm'
num_epoch = 20
#k_fold = 5                  #k-fold value
lr = 0.001
bidirection = False
if_attention = True
gra_clip = True
save_model_path = 'drive/My Drive/Bioinformatic/LSTM1_a'

#perm_seq, perm_label,perm_length = data_shuffle(enc_sentences, np.array(Labels), sentence_lengths, random_seed = 123)
#enc_truncated,# enc_length = truncate(perm_seq, perm_length)


full_run = run(vocab,batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip, kernel_size)
TL_full, Tacc_full = full_run.train_full_data(enc_length, enc_truncated, perm_label, bidirection, if_attention, model_name,save_model_path)


#Run_lstm0 = run(batch_size, output_size, hidden_size, vocab_size, embedding_size, num_epoch, lr, gra_clip)
#TL0, Tacc0, VL0, Vacc0 = Run_lstm0.lets_go(sentence_lengths, enc_sentences, Labels, k_fold, bidirection, if_attention ,save_model_path)

testing

In [0]:
enc_test = test_enc_sentences
enc_test_length = test_sentence_lengths
at = full_run.testing(enc_test, enc_test_length)

In [0]:
full_run.print_frame()

In [0]:
attention = at.cpu().detach().numpy()

In [0]:
adjust_test = np.zeros(attention.shape)

for i in range(attention.shape[0]):
    num_nonzero = np.count_nonzero(attention[i])
    if num_nonzero <2000:
        adjust_test[i,: int(num_nonzero/2)] = attention[i, : int(num_nonzero/2)]
        adjust_test[i,-(num_nonzero - int(num_nonzero/2)):] = attention[i,int(num_nonzero/2):num_nonzero]



In [0]:
fig, ax = plt.subplots(figsize=(10, 10))
img = ax.imshow(adjust_test, interpolation='nearest',aspect='auto', cmap = 'binary', vmin=0, vmax=0.03);
#img2 = ax.imshow(adjust_se, interpolation='nearest', aspect='auto', cmap = 'binary', vmin = 0, vmax = 0.03);
ax.set_title('testing attention')
ax.set_xlabel('Position')
ax.set_ylabel('Sequences number')
fig.colorbar(img,orientation='vertical')

In [0]:
peptide = []
threshold = 0.01
for i in range(attention.shape[0]):
    temp = ''
    for j in range(attention.shape[1]):
        if attention[i,j] == 0:
            break
        if attention[i,j] >threshold:
            temp += test['seq'][i][j]
        else:
            temp += '_'
    peptide.append(temp)



In [0]:
for i in range(len(peptide)):
    print(test['seq'][i] + '\n' + peptide[i])